In [1]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.callbacks import Callback, ModelCheckpoint,EarlyStopping
import pandas as pd 
import numpy as numpy 

In [2]:
###########Data###############

In [3]:
class_num = pd.read_csv(r'D:\\shuwa\\pj15.5\\class_name.csv')

In [4]:
# weight = pd.read_csv(r'D:\\shuwa\\pj15.5\\alphabet\\alphabet.csv',header=None)
# label = pd.read_csv(r'D:\\shuwa\\pj15.5\\alphabet\\alphabet_class.csv')
# weight2 = pd.read_csv(r'D:\\shuwa\\pj15.5\\alphabet\\alphabet5.csv',header=None)
# label2 = pd.read_csv(r'D:\\shuwa\\pj15.5\\alphabet\\alphabet5_class.csv')
# weight3 = pd.read_csv(r'D:\\shuwa\\pj15.5\\alphabet\\alphabet3.csv',header=None)
# label3 = pd.read_csv(r'D:\\shuwa\\pj15.5\\alphabet\\alphabet3_class.csv')
weight1 = pd.read_csv(r'D:\\shuwa\\pj15.5\\Hieu\\HIEU.csv',header=None)
label1 = pd.read_csv(r'D:\\shuwa\\pj15.5\\Hieu\\HIEU_class.csv')
weight2 = pd.read_csv(r'D:\\shuwa\\pj15.5\\Hieu\\J.csv',header=None)
label2 = pd.read_csv(r'D:\\shuwa\\pj15.5\\Hieu\\J_class.csv')
weight3 = pd.read_csv(r'D:\\shuwa\\pj15.5\\motion\\coords.csv',header=None)
label3 = pd.read_csv(r'D:\\shuwa\\pj15.5\\motion\\coords_class.csv')
weight4 = pd.read_csv(r'D:\\shuwa\\pj15.5\\motion\\coords2.csv',header=None)
label4 = pd.read_csv(r'D:\\shuwa\\pj15.5\\motion\\coords2_class.csv')
weight = pd.concat([weight1,weight2,weight3,weight4])
label = pd.concat([label1,label2,label3,label4])

In [5]:
y = label.to_numpy()
y_label = np.array([np.where(class_num == i)[0] for i in y])

In [6]:
y_label

array([[7],
       [7],
       [7],
       ...,
       [5],
       [5],
       [5]], dtype=int64)

In [7]:
def datalabel(weight):
    data = weight.to_numpy()
    
    x = []
    for i in range(0,len(y)):
        a = []
        for j in range(0,16):
            a.append(data[j+16*i])
        x.append(a)
    return np.array(x)

In [8]:
weight_hand_left = weight.iloc[:,:63]
weight_hand_right = weight.iloc[:,63:126]
weight_face = weight.iloc[:,126:]

In [9]:
x_hand_right = datalabel(weight_hand_right)
x_hand_left = datalabel(weight_hand_left)
x_face = datalabel(weight_face)

In [10]:
from sklearn.model_selection import train_test_split
X_hand_right_train,X_hand_right_test,X_hand_left_train,X_hand_left_test,X_face_train,X_face_test,Y_train,Y_test = train_test_split(x_hand_right,x_hand_left,x_face,y_label,                                                                                                                                                     test_size=0.3,random_state=1234)

In [11]:
###########Model##############

In [12]:
def buildmodel():
    ##Input##
    left_hand = layers.Input(shape=(16,63))
    right_hand = layers.Input(shape=(16,63))
    face = layers.Input(shape=(16,1404))
    #Mid Layer##
    ###Face Branch###
    Face = layers.Flatten()(face)
    Face = layers.Reshape((-1,22464))(Face)
    Face = layers.LSTM(468,kernel_regularizer=tf.keras.regularizers.l1(0.01),
                            activity_regularizer=tf.keras.regularizers.l2(0.01),
                            activation='tanh',return_sequences=True)(Face)
    ###Right Hand###
    RH = layers.Flatten()(right_hand)
    RH = layers.Reshape((-1,1008))(RH)
    RH = layers.LSTM(21,kernel_regularizer=tf.keras.regularizers.l1(0.01),
                        activity_regularizer=tf.keras.regularizers.l2(0.01),
                        activation='tanh',return_sequences=True)(RH)
    ###Left Hand###
    LH = layers.Flatten()(left_hand)
    LH = layers.Reshape((-1,1008))(LH)
    LH = layers.LSTM(21,kernel_regularizer=tf.keras.regularizers.l1(0.01),
                        activity_regularizer=tf.keras.regularizers.l2(0.01),
                        activation='tanh',return_sequences=True)(LH)
    ###Combine###
    combined = layers.concatenate([LH,RH,Face])
    shaped = layers.Reshape((-1,510))(combined)
    results = layers.LSTM(102,activation='sigmoid')(shaped)
    results = layers.Dropout(0.1)(results)
    results = layers.Dense(len(class_num),activation='softmax')(results)
    model = models.Model([left_hand, right_hand, face],results)
    return model
import os
from keras.models import load_model

if os.path.exists("./model.h5"):
    model = load_model("./model.h5")
else:
    model = buildmodel()

In [13]:
cp_callback = ModelCheckpoint(
    './', verbose=1, save_weights_only=False)
es_callback = EarlyStopping(patience=40, verbose=1)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )
checkpoint = ModelCheckpoint('../models/model.h5', monitor='val_acc', verbose=1, mode = 'max', save_best_only=True, save_weights_only=False, period=1)

In [14]:
model.fit(
    [X_hand_left_train,X_hand_right_train,X_face_train],
    Y_train,
    epochs=100,
    batch_size=64,
    validation_data=([X_hand_left_test,X_hand_right_test,X_face_test], Y_test),
    callbacks=[cp_callback,es_callback]
)

ving model to .\
INFO:tensorflow:Assets written to: .\assets
INFO:tensorflow:Assets written to: .\assets
Epoch 68/100
66/66 [==============================] - 23s 352ms/step - loss: 48.1091 - accuracy: 0.9105 - val_loss: 48.7953 - val_accuracy: 0.9103

Epoch 00068: saving model to .\
INFO:tensorflow:Assets written to: .\assets
INFO:tensorflow:Assets written to: .\assets
Epoch 69/100
66/66 [==============================] - 25s 378ms/step - loss: 48.0436 - accuracy: 0.9198 - val_loss: 48.6243 - val_accuracy: 0.9341

Epoch 00069: saving model to .\
INFO:tensorflow:Assets written to: .\assets
INFO:tensorflow:Assets written to: .\assets
Epoch 70/100
66/66 [==============================] - 26s 398ms/step - loss: 47.9171 - accuracy: 0.9290 - val_loss: 47.8955 - val_accuracy: 0.9291

Epoch 00070: saving model to .\
INFO:tensorflow:Assets written to: .\assets
INFO:tensorflow:Assets written to: .\assets
Epoch 71/100
66/66 [==============================] - 25s 374ms/step - loss: 47.9949 - accu

In [15]:
model.save("./model.h5", include_optimizer=False)